In [7]:
import pandas as pd
import os
import json
import re
from tqdm import tqdm

In [ ]:
pickle_file = "hsbc_site_20250220.limited.pickle"
df:pd.DataFrame=pd.read_pickle(pickle_file)
df.head(1)

,catalog,files_links,url,title,content
0,accounts,[/content/dam/hsbc/hk/docs/premier/accounts-co...,https://www.hsbc.com.hk/accounts/,"Bank Accounts | Savings, Current, Time Deposit...","<!DOCTYPE html><html lang=""en-hk"" dir=""ltr"" it..."


In [ ]:
from urllib.parse import ParseResult, urlparse
def extract_base_url(url):
    parsed_url:ParseResult = urlparse(url)
    base_url = parsed_url.scheme + "://" + parsed_url.netloc
    return base_url
def remove_first_url(url_string:str):
    second_https_start_index = url_string.find("https://", 1) 
    if second_https_start_index != -1:
        return url_string[second_https_start_index:]
    else:
        return url_string

In [ ]:
files_links=[]
for index, row in df.iterrows():
    files_links_list = row['files_links']
    catalog = row['catalog']
    if isinstance(files_links_list, list):
        for link in files_links_list:
            base_url=extract_base_url(row["url"])
            full_link = f"{base_url}{link}"
            full_link = remove_first_url(full_link)
            files_links.append(full_link)
unique_links_set = set(files_links)
unique_links_list = list(unique_links_set)

In [ ]:
skipping_links = []

In [ ]:
import requests
download_directory = "transformed_limited_hsbc_site_20250220/datasets/default"
for full_link in tqdm(unique_links_list):
    if full_link in skipping_links:
        continue
    parsed_url:ParseResult = urlparse(full_link)
    path_segments = parsed_url.path.split('/')
    if len(path_segments) > 1:
        catalog=path_segments[1]
    else:
        netloc = parsed_url.netloc
        domain_parts = netloc.split('.')
        if len(domain_parts) > 1:
            catalog = domain_parts[0]
        else:
            catalog = netloc
    file_path = parsed_url.path

    if file_path.startswith('/'):
        file_path = file_path[1:]
    save_path = os.path.join(download_directory, file_path)
    if os.path.exists(save_path):
        continue
    try:
        response = requests.get(full_link, stream=True) 
        response.raise_for_status() 

        if not os.path.exists(os.path.dirname(save_path)):
            os.makedirs(os.path.dirname(save_path), exist_ok=True)

        if os.path.exists(save_path):
            save_path=f"{catalog}/{save_path}"
        try:
            with open(save_path, 'wb') as file: 
                for chunk in response.iter_content(chunk_size=8192):
                    file.write(chunk)
        except Exception as e:
            print(f"Error saving {full_link} to {save_path}: {e}")

        # print(f"Downloaded: {full_link} -> Saved to: {save_path}")

    except requests.exceptions.RequestException as e:
        print(f"Error downloading {full_link}: {e}")
        skipping_links.append(full_link)

print("Download process completed.")

 27%|██▋       | 871/3216 [00:01<00:04, 489.93it/s]

Error saving https://www.hsbc.com.hk/content/dam/hsbc/en/docs/wealth-insights/market-outlook/global-event/cautious-fed-pauses-awaiting-further-evidence-of-lower-inflation-before-continuing-slower-easing-cycle-tc.pdf to transformed_hsbc_site_20250220/datasets/default\content/dam/hsbc/en/docs/wealth-insights/market-outlook/global-event/cautious-fed-pauses-awaiting-further-evidence-of-lower-inflation-before-continuing-slower-easing-cycle-tc.pdf: [Errno 2] No such file or directory: 'transformed_hsbc_site_20250220/datasets/default\\content/dam/hsbc/en/docs/wealth-insights/market-outlook/global-event/cautious-fed-pauses-awaiting-further-evidence-of-lower-inflation-before-continuing-slower-easing-cycle-tc.pdf'


 28%|██▊       | 913/3216 [01:16<14:29,  2.65it/s] 

Error downloading https://www.hsbc.com.hkhttps://www.hsbc.com.hk/content/dam/hsbc/hk/vam/pdf/2024_11_PIL_W11_Online_TnC_PTL_1129_R1.pdf: HTTPSConnectionPool(host='www.hsbc.com.hkhttps', port=443): Max retries exceeded with url: /www.hsbc.com.hk/content/dam/hsbc/hk/vam/pdf/2024_11_PIL_W11_Online_TnC_PTL_1129_R1.pdf (Caused by NameResolutionError("<urllib3.connection.HTTPSConnection object at 0x000001F022AB4E30>: Failed to resolve 'www.hsbc.com.hkhttps' ([Errno 11001] getaddrinfo failed)"))


 29%|██▊       | 923/3216 [01:43<38:44,  1.01s/it]

...

Error downloading https://www.hsbc.com.hkhttps://www.hsbc.com.hk/content/dam/hsbc/hk/vam/pdf/EBSLeaflet_EXE_TC_2025.pdf: HTTPSConnectionPool(host='www.hsbc.com.hkhttps', port=443): Max retries exceeded with url: /www.hsbc.com.hk/content/dam/hsbc/hk/vam/pdf/EBSLeaflet_EXE_TC_2025.pdf (Caused by NameResolutionError("<urllib3.connection.HTTPSConnection object at 0x000001F022AC2EA0>: Failed to resolve 'www.hsbc.com.hkhttps' ([Errno 11001] getaddrinfo failed)"))


100%|█████████▉| 3200/3216 [1:27:26<00:56,  3.54s/it]

Error downloading https://www.hsbc.com.hkhttps://cdn.hsbc.com.hk/content/dam/hsbc/hk/tc/docs/investments/stocks/top-trader-club/local-securities-product-fact-sheet.pdf: HTTPSConnectionPool(host='www.hsbc.com.hkhttps', port=443): Max retries exceeded with url: /cdn.hsbc.com.hk/content/dam/hsbc/hk/tc/docs/investments/stocks/top-trader-club/local-securities-product-fact-sheet.pdf (Caused by NameResolutionError("<urllib3.connection.HTTPSConnection object at 0x000001F022AC1B80>: Failed to resolve 'www.hsbc.com.hkhttps' ([Errno 11001] getaddrinfo failed)"))


100%|██████████| 3216/3216 [1:27:54<00:00,  1.64s/it]

Error downloading https://www.hsbc.com.hk/content/dam/hsbc/hk/docs/mpf/CPFexample_c.pdf: 404 Client Error: Not Found for url: https://www.hsbc.com.hk/content/dam/hsbc/hk/docs/mpf/CPFexample_c.pdf
Download process completed.


In [25]:
os.path.exists(os.path.dirname(save_path)),os.path.exists(save_path)

(True, False)